<a href="https://colab.research.google.com/github/nuno-nunes/bombeiros-jlle/blob/master/mapa_bombeiros_folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importando arquivos que serão utilizados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Opção para trabalhar offline

In [ ]:
#from google.colab import files
#uploaded = files.upload()
#file = '/content/drive/My Drive/python/bombeiros/db-bombeiros-jlle-jun-20-limpo-legenda - mapa-acidentes-2020 (1).csv'

###Bibliotecas


In [ ]:
import pandas as pd
import folium
import re
import os
import branca as bc
import seaborn as sns
from folium.plugins import BeautifyIcon
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import time
import glob
from google.colab import files

###Lendo e verificando os dados

In [ ]:
file = '/content/drive/My Drive/python/bombeiros/db-bombeiros-1-sem-20-limpo-coordenadas - mapa-acidentes-att-2.csv'
df_import = pd.read_csv(file)
df_import

,Index,data,cor,favoritos,hashtags,link,mentions,retweets,texto,to,tweetId,ano,dia_mes,semana,mes,hora,endereco,descricao,tipo_ocorrência,bairro,latitude,longitude,endereco_completo
0,1,2020-07-13T22:18:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 19h15 - Carro x Moto Local...,NaN,1282801564063866880,2020,13/07,29,7,19h15,Rua Marcos Joao Serafim,Carro x Moto,Acidente de Trânsito,Nova Brasília,NaN,NaN,"Rua Marcos Joao Serafim , Nova Brasília, Joinv..."
1,2,2020-07-13T22:00:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 18h59 - Carro x Moto Local...,NaN,1282797036685922304,2020,13/07,29,7,18h59,"Rua Luiz Bachtold, 596",Carro x Moto,Acidente de Trânsito,Costa e Silva,-26.265673,-48.881747,"Rua Luiz Bachtold, 596 , Costa e Silva, Joinvi..."
2,7,2020-07-13T14:51:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 11h35 - Carro x Moto Local...,NaN,1282689210273673228,2020,13/07,29,7,11h35,Rua Minas Gerais,Carro x Moto,Acidente de Trânsito,Nova Brasília,-26.331693,-48.869583,"Rua Minas Gerais , Nova Brasília, Joinville, S..."
3,11,2020-07-13T00:05:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 09h06 - Carro x Moto Local...,NaN,1282647435949625346,2020,13/07,29,7,09h06,"Rua Guilherme, 518",Carro x Moto,Acidente de Trânsito,Costa e Silva,-26.279705,-48.865146,"Rua Guilherme, 518 , Costa e Silva, Joinville,..."
4,13,2020-07-13T08:27:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 05h08 - Queda de Moto Loca...,NaN,1282592573396262913,2020,13/07,29,7,05h08,"Rua Icarai, 448",Queda de Moto,Acidente de Trânsito,Floresta,-26.335977,-48.850719,"Rua Icarai, 448 , Floresta, Joinville, Santa C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,3405,2020-01-03T18:28:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1213...,NaN,0,Ocorrência: 03/01 - 15h24 - Queda de Moto Loca...,NaN,1213165150020218880,2020,03/01,1,1,15h24,"Rua Tuiuti, 712",Queda de Moto,Acidente de Trânsito,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 712, Aventureiro, Joinville, Santa..."
1297,3408,2020-01-03T13:28:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1213...,NaN,0,Ocorrência: 03/01 - 10h26 - Carro x Moto Local...,NaN,1213089682969501697,2020,03/01,1,1,10h26,"Rua Monsenhor Gercino, 6975",Carro x Moto,Acidente de Trânsito,Itaum,-26.330427,-48.837613,"Rua Monsenhor Gercino, 6975, Itaum, Joinville,..."
1298,3413,2020-01-02T21:55:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1212...,NaN,0,Ocorrência: 02/01 - 18h52 - Queda de Moto Loca...,NaN,1212854888532758529,2020,02/01,1,1,18h52,"Rua Uruguai, 250",Queda de Moto,Acidente de Trânsito,Itaum,-26.328205,-48.840306,"Rua Uruguai, 250, Itaum, Joinville, Santa Cata..."
1299,3417,2020-01-02T16:36:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1212...,NaN,0,Ocorrência: 02/01 - 13h20 - Queda de Moto Loca...,NaN,1212774590184865793,2020,02/01,1,1,13h20,"Rua Tuiuti, 2936",Queda de Moto,Acidente de Trânsito,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 2936, Aventureiro, Joinville, Sant..."


###Filtrando os dados

In [ ]:
df = df_import.copy()
#df.drop(df.columns[0], axis=1)
#pontos com problema de localização
#2579
#2358
#495

df.rename(columns={ df.columns[0]: "index" }, inplace = True)
df

,index,data,cor,favoritos,hashtags,link,mentions,retweets,texto,to,tweetId,ano,dia_mes,semana,mes,hora,endereco,descricao,tipo_ocorrência,bairro,latitude,longitude,endereco_completo
0,1,2020-07-13T22:18:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 19h15 - Carro x Moto Local...,NaN,1282801564063866880,2020,13/07,29,7,19h15,Rua Marcos Joao Serafim,Carro x Moto,Acidente de Trânsito,Nova Brasília,NaN,NaN,"Rua Marcos Joao Serafim , Nova Brasília, Joinv..."
1,2,2020-07-13T22:00:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 18h59 - Carro x Moto Local...,NaN,1282797036685922304,2020,13/07,29,7,18h59,"Rua Luiz Bachtold, 596",Carro x Moto,Acidente de Trânsito,Costa e Silva,-26.265673,-48.881747,"Rua Luiz Bachtold, 596 , Costa e Silva, Joinvi..."
2,7,2020-07-13T14:51:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 11h35 - Carro x Moto Local...,NaN,1282689210273673228,2020,13/07,29,7,11h35,Rua Minas Gerais,Carro x Moto,Acidente de Trânsito,Nova Brasília,-26.331693,-48.869583,"Rua Minas Gerais , Nova Brasília, Joinville, S..."
3,11,2020-07-13T00:05:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 09h06 - Carro x Moto Local...,NaN,1282647435949625346,2020,13/07,29,7,09h06,"Rua Guilherme, 518",Carro x Moto,Acidente de Trânsito,Costa e Silva,-26.279705,-48.865146,"Rua Guilherme, 518 , Costa e Silva, Joinville,..."
4,13,2020-07-13T08:27:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1282...,NaN,0,Ocorrência: 13/07 - 05h08 - Queda de Moto Loca...,NaN,1282592573396262913,2020,13/07,29,7,05h08,"Rua Icarai, 448",Queda de Moto,Acidente de Trânsito,Floresta,-26.335977,-48.850719,"Rua Icarai, 448 , Floresta, Joinville, Santa C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,3405,2020-01-03T18:28:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1213...,NaN,0,Ocorrência: 03/01 - 15h24 - Queda de Moto Loca...,NaN,1213165150020218880,2020,03/01,1,1,15h24,"Rua Tuiuti, 712",Queda de Moto,Acidente de Trânsito,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 712, Aventureiro, Joinville, Santa..."
1297,3408,2020-01-03T13:28:00Z,#e55367,1,NaN,https://twitter.com/bvsc_joinville/status/1213...,NaN,0,Ocorrência: 03/01 - 10h26 - Carro x Moto Local...,NaN,1213089682969501697,2020,03/01,1,1,10h26,"Rua Monsenhor Gercino, 6975",Carro x Moto,Acidente de Trânsito,Itaum,-26.330427,-48.837613,"Rua Monsenhor Gercino, 6975, Itaum, Joinville,..."
1298,3413,2020-01-02T21:55:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1212...,NaN,0,Ocorrência: 02/01 - 18h52 - Queda de Moto Loca...,NaN,1212854888532758529,2020,02/01,1,1,18h52,"Rua Uruguai, 250",Queda de Moto,Acidente de Trânsito,Itaum,-26.328205,-48.840306,"Rua Uruguai, 250, Itaum, Joinville, Santa Cata..."
1299,3417,2020-01-02T16:36:00Z,#e55367,0,NaN,https://twitter.com/bvsc_joinville/status/1212...,NaN,0,Ocorrência: 02/01 - 13h20 - Queda de Moto Loca...,NaN,1212774590184865793,2020,02/01,1,1,13h20,"Rua Tuiuti, 2936",Queda de Moto,Acidente de Trânsito,Aventureiro,-26.239733,-48.818731,"Rua Tuiuti, 2936, Aventureiro, Joinville, Sant..."


###Inserindo coordenadas geográficas nos dados

###Layout do mapa


In [ ]:
#Cria um mapa com todos os estilos e um seletor para escolher melhor opção

# map
jlle_map_layout = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=11)

# add tiles to map
folium.raster_layers.TileLayer('Open Street Map').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('Stamen Toner').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map_layout)
folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(jlle_map_layout)

# add layer control to show different maps
folium.LayerControl().add_to(jlle_map_layout)

# display map
jlle_map_layout

In [ ]:
#Layout escolhido
jlle_map = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map)
jlle_map

###Análise: mapa dos tipos de acidentes


Mapa 1 - Tipos de acidentes

In [ ]:
#df['cor'] = df['legenda']

#'Moto' = "#E55367"
#'Moto x Carro'= "#DFB235"
#'Carro' = "#E07829"
#'Outros' = "#8D6D68"

#df['cor'] = df.cor.str.replace(r'(Moto$)', 'red')
#df['cor'] = df.cor.str.replace(r'(Moto x Carro)', 'purple')
#df['cor'] = df.cor.str.replace(r'(Carro$)', 'cadetblue')
#df['cor'] = df.cor.str.replace(r'(Outros)', 'darkgreen')

#dicionario_cores = {
#		"Moto" : "red",
#		"Moto x Carro" : 'purple',
#		"Carro" : "cadetblue",
#		"Outro" : "darkgree"
#	}


#df

In [ ]:
df=df.dropna(subset=['longitude'])
df=df.dropna(subset=['latitude'])
len(df)
df.head(0)

,index,data,cor,favoritos,hashtags,link,mentions,retweets,texto,to,tweetId,ano,dia_mes,semana,mes,hora,endereco,descricao,tipo_ocorrência,bairro,latitude,longitude,endereco_completo


In [ ]:
dfs_semanas = []
for i in range(1,28):
  #name_df = "df_semana_" + str(i)
  df_temp = df[df.semana == i]
  #name = df_temp
  dfs_semanas.append(df_temp)
  #dfs = {name: split_all_weekds(df, name) for name in name_df}
  #name = df_temp.copy()




dfs_semanas 

[      index  ...                                  endereco_completo
 1290   3381  ...  Rua Anita Garibaldi, 2250, Anita Garibaldi, Jo...
 1291   3388  ...  Rua Quinze de Novembro, 4315, Centro, Joinvill...
 1293   3400  ...  Estrada da Ilha, 2040, Pirabeiraba, Joinville,...
 1294   3403  ...  Rua Paulo Roberto Anastácio, 291, Paranaguamir...
 1295   3404  ...  Rua Teresopolis, 254, Guanabara, Joinville, Sa...
 1296   3405  ...  Rua Tuiuti, 712, Aventureiro, Joinville, Santa...
 1297   3408  ...  Rua Monsenhor Gercino, 6975, Itaum, Joinville,...
 1298   3413  ...  Rua Uruguai, 250, Itaum, Joinville, Santa Cata...
 1299   3417  ...  Rua Tuiuti, 2936, Aventureiro, Joinville, Sant...
 
 [9 rows x 23 columns],
       index  ...                                  endereco_completo
 1247   3256  ...  Rua Tabatinga, Jardim Iririú, Joinville, Santa...
 1248   3259  ...  Rua Luiza Deranholi Koschnik, 263, Aventureiro...
 1250   3266  ...  Rua Monsenhor Gercino, 17, Itaum, Joinville, S...
 1251   

In [ ]:
#cria contador para nomear arquivo e define 2 digitos
padding = 2
counter = 00

#tabela para os dados ficarem cumulativos no loop
df_mestre = pd.DataFrame()


jlle_map = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)

folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map)

for i in dfs_semanas:
  df_mestre = pd.concat([df_mestre, i], axis=0, sort=True) 
  for _, case in df_mestre.iterrows():
    popup_text = 'Tipo: {:} Data: {:} \
    Endereco: {:} Index: {:}'
    popup_text = popup_text.format(case['descricao'],
                                  case['dia_mes'],
                                  case['endereco'],
                                  case['index'])

    folium.Circle(location=[case['latitude'], 
                                  case['longitude']],
                                  radius=35,
                                  color=case['cor'],
                                  popup=popup_text,
                                  fill=True).add_to(jlle_map)
  counter = counter + 1
  file_name = 'jlle_map_acidentes_' + str(counter).zfill(padding) + ".html"
  jlle_map.save(file_name)

#folium.CircleMarker.add_to(jlle_map)
#locais = df[['latitude', 'longitude']].values.tolist()



In [ ]:
curr_dir = os.getcwd()
#print(curr_dir)
files_ =glob.glob(curr_dir+"\*.html")
files_ =glob.glob(curr_dir+"/jlle_map_acidentes_*.html")

for file in files_:
  files.download(file) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Analise 2 - acidentes antes em relação a pandemia


Separa por semanas 


In [ ]:
df['semana_cor'] = df['semana'].copy()
df_antes_quarentena = df[df.semana <= 10]
df_durante_quarentena = df[(df["semana"] <=14) & (df["semana"] >=11)]
df_depois_quarentena = df[df.semana >= 15]

df_depois_quarentena

In [ ]:
df['semana_cor'] = df['semana'].copy()

df.loc[569:1062,'semana_cor'] = "cadetblue"

df.loc[441:568,'semana_cor'] = "red"

df.loc[1:437,'semana_cor'] = "orange"

df

até 09/03 até <=10

até 30/03 11<=>14

a partir de 1/04 >=15

year_2015_data = crime_rates_per_year_data[crime_rates_per_year_data.YEAR == 2015]

In [ ]:
jlle_map_corona = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)

folium.raster_layers.TileLayer('CartoDB Positron').add_to(jlle_map_corona)

for _, case in df.iterrows():
  popup_text = 'Tipo: {:} Data: {:} \
  Endereco: {:} Index: {:}'
  popup_text = popup_text.format(case['descricao'],
                                 case['data2'],
                                 case['endereco'],
                                 case['index'])

  folium.Circle(location=[case['latitude'], 
                                case['longitude']],
                                radius=35,
                                color=case['semana_cor'],
                                popup=popup_text,
                                fill=True).add_to(jlle_map_corona)

#Circle().add_to(jlle_map_corona)
#locais = df[['latitude', 'longitude']].values.tolist()
jlle_map_corona
jlle_map_corona.save('jlle_map_corona2.html')

### Mapa Calor - Acidentes 


In [ ]:
df.to_csv('bombeiros-coordenadas.csv')

In [ ]:
calor_map = folium.Map(location=[-26.294052, -48.841550], 
                              zoom_start=12)

folium.raster_layers.TileLayer('CartoDB Positron').add_to(calor_map)

#folium.CircleMarker.add_to(jlle_map)
locais = df[['latitude', 'longitude']].values.tolist()
HeatMap(locais, radius=20).add_to(calor_map)
calor_map
calor_map.save("calor_map.html") 